# 2주차까지 진행한 상황

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # 모든 열을 출력한다.

# 기본 시각화 라이브러리 임포트
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import seaborn as sns 
from matplotlib import rc

# 기본 한글 폰트로 설정
rc("font", family="AppleGothic")  # 맥의 경우
# rc("font", family="Malgun Gothic")  # 윈도우의 경우

# 음수 부호까지 제대로 표시되도록 설정 추가
plt.rcParams['axes.unicode_minus'] = False

# 경고 메시지 무시
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/winter 24/project7/train.csv')
test_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/winter 24/project7/test.csv')
sample_submission_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/winter 24/project7/sample_submission.csv')

In [3]:
# 의미없는 변수 제거 [index, FLAG_MOBIL]
train_df.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
# test_df.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

In [4]:
# 결측치 처리 [1번째 방법]
train_df.fillna('NaN', inplace=True)
# test_df.fillna('NaN', inplace=True)

# 결과 확인
print(train_df['occyp_type'].isnull().sum())  # 남아있는 결측값 개수 확인

0


In [5]:
# 결측치 처리 [2번째 방법]

# "occyp_type" 열의 값에 대한 비율 계산
occyp_type_ratios = train_df["occyp_type"].value_counts(normalize=True)

# 결측값을 채울 함수 정의
def fill_missing_occyp_type(row):
    if pd.isnull(row['occyp_type']):
        return occyp_type_ratios.sample(1, weights=occyp_type_ratios).index[0]
    else:
        return row['occyp_type']

# 'occyp_type' 열에 적용
train_df['occyp_type'] = train_df.apply(fill_missing_occyp_type, axis=1)

# 결과 확인
print(train_df['occyp_type'].isnull().sum())  # 남아있는 결측값 개수 확인

0


In [6]:
# 결측치 처리 [3번째 방법]

# 상관관계 파악 후 결측치 변수와 관련성이 높은 변수를 기준으로 결측치 처리 해보기!

In [7]:
# 이상치 처리 
train_df = train_df[(train_df['family_size'] <= 7)] # 가족 수가 7 초과인 것들 삭제]
train_df = train_df[(train_df['child_num'] <= 5)] # 자녀 수가 5 초과인 것들 삭제
train_df = train_df.reset_index(drop=True) # 새롭게 초기화 배열

In [8]:
# 변수 변환 [양수는 0으로 바꿔 고용되지 않은 상태로 변환]
train_df['DAYS_EMPLOYED'] = train_df['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
# test['DAYS_EMPLOYED'] = test['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)

In [9]:
# 변수 변환 [음수값 -> 양수 변환]
# 날짜 변수 : DAYS_BIRTH, begin_month, DAYS_EMPLOYED

dates = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for date in dates:
    train_df[date]=np.abs(train_df[date])
    # test_df[date]=np.abs(test_df[date])

In [10]:
# 구간화 함수 제작
def make_bin(df, variable, n):
    
    data = df
    count, bin_dividers = np.histogram(data[variable], bins=n)
    bin_names=[str(i) for i in range(n)]
    data['%s_bin' % variable] = pd.cut(x=data[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    data['%s_bin' % variable] = pd.factorize(data['%s_bin' % variable])[0]
    print(data['%s_bin' % variable], '\n\n')
    
    return data

In [11]:
# 파생변수 [함수화]
# numeric 변수는 최대한 다양한 특징을 보일 수 있도록 생성 + category 변수는 여러가지를 조합해 보기!
# [train_df,test_df] 대입하면 test까지 한 번에 가능!!

for df in [train_df] :
    
    # [1] before_EMPLOYED : 고용되기 전까지의 일수 (고용비율)
    df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
    
    # [2] income_total_befofeEMP_ratio : 연간 소득 중 고용되기 전까지의 비율
    df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']

    # [3] before_EMPLOYED_year : 고용되지 전까지의 년
    df['before_EMPLOYED_year'] = df['before_EMPLOYED'] // 365

    # [4] before_EMPLOYED_month : 고용된 몇 월
    df['before_EMPLOYED_month'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)

    # [5] before_EMPLOYED_week : 고용된 몇째 주
    df['before_EMPLOYED_week'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)
    
    # [6] Age : 나이
    df['Age'] = df['DAYS_BIRTH'] // 365
    
    # [7] DAYS_BIRTH_month : 태어난 월 (출생연도의 n월)
    df['DAYS_BIRTH_month'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
    
    # [8] DAYS_BIRTH_week : 태어난 주 (출생연도의 n주차)
    df['DAYS_BIRTH_week'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)

    # [9] EMPLOYED : 근속연수
    df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
    
    # [10] DAYS_EMPLOYED_month : 고용된 달 (고용연도의 n월)
    df['DAYS_EMPLOYED_month'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)

    # [11] DAYS_EMPLOYED_week : 고용된 주 (고용연도의 n주차)
    df['DAYS_EMPLOYED_week'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

    # [12] ability : 소득 / (근무일수)
    df['ability'] = df['income_total'] / (df['DAYS_EMPLOYED'])
    
    # [13] income_mean : 소득 / 가족 수 (인당 평균 부양비)
    df['income_mean'] = df['income_total'] / df['family_size']

    # [14] : raito_EMPLOYED (인생의 일한 비율)
    df['raito_EMPLOYED'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

    # [15] : begin_year (신용카드 발급 년도)
    df['begin_year'] = df['begin_month'] // 12

    # [16] 만 단위로 생성
    df['income_total'] = df['income_total']/10000

    # [17] ID 생성 {1번째 방법}
    df['personal_id'] = df['gender'] + "_" + df['DAYS_BIRTH'].astype(str) + "_" + df['income_total'].astype(str) + df['income_type'].astype(str)

    # [18] ID 생성 {2번째 방법}
    df['ID'] = \
    df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
    df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
    df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
    df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
    df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
    df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
    df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
    df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)